## Countdown Numbers Game

###  Overview and explanation of the Countdown Numbers Game

**Rules:**

1. Game generates 6 random numbers
2. The game generate target number
3. The contestant has to get to the target number with any amount of numbers from the 6 random numbers, they can only use each number once. They don't have to use all the random numbers.

### Example
*Here's an example of the game in action. In this instance, one number was selected from the large set, and the rest from the small set.* [[1](#section1)]

$ \{ 50 , 8 , 3 , 7 , 2 , 10 \} $

*The randomly selected target was **556***.

*There are multiple ways to solve this. The smallest solution requires just four numbers:*

$ (50 × 10) + (8 × 7) = 556 $

*More complex solutuion:*

$ (((50 - 7) × 3) + 10) × 8) ÷ 2 = 556 $

### Discussion of the complexity of the Countdown Numbers Game

* *Is it possible*
* *What are possible ways of doing it if it can be done* 
* *If it's more than one way, can you find different ways*

### Code

## Itertools
    


*Example from [here](https://realpython.com/python-itertools/)*

*Here’s a common interview-style problem:
You have three 20 dollar bills, five 10 dollar bills, two 5 dollar bills, and five 1 dollar bills. How many ways can you make change for a 100 dollar bill?
To “brute force” this problem, you just start listing off the ways there are to choose one bill from your wallet, check whether any of these makes change for 100, then list the ways to pick two bills from your wallet, check again, and so on and so forth.*

In [1]:
import itertools as it

bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]

A choice of $k$ things from a set of n things is called a **combination**

The *itertools.combinations()* function takes two arguments—an iterable inputs and a positive integer n—and produces an iterator over tuples of all combinations of $n$ elements in inputs.

In [2]:
#list(it.combinations(bills, 3))

To solve the problem, you can loop over the positive integers from 1 to len(bills), then check which combinations of each size add up to $100:

In [3]:
makes_100 = []
for n in range(1, len(bills) + 1):
    for combination in it.combinations(bills, n):
        if sum(combination) == 100:
            makes_100.append(combination)

In [4]:
print(makes_100)

[(20, 20, 20, 10, 10, 10, 10), (20, 20, 20, 10, 10, 10, 10), (20, 20, 20, 10, 10, 10, 10), (20, 20, 20, 10, 10, 10, 10), (20, 20, 20, 10, 10, 10, 10), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 5), (20, 20, 10, 10, 10, 10, 10, 5, 5), (20, 20, 10, 10, 10, 10, 10, 5, 5), (20, 20, 10, 10, 10, 10, 10, 5, 5), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1), (20, 20, 20, 10, 10, 10, 

In [5]:
set(makes_100)

{(20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 10)}

## Countdown Numbers
***
* *You're not allowed to have a negative number*
* *You cannot have a fraction e.g 50/6*

In [6]:
# Permutations and combinations
import itertools as it
# RAndom number generation
import random
# Operators as function
import operator

## Simulate a Game

In [7]:
# Randomly create a game
def new_numbers_game(no_large=None):
    """Returns six numbers and a target number representing a Countdown Numbers Game"""
    #If no_large is None, then randomly pick value between 0 and 4 inclusive
    if no_large is None:
        # Randomly set a value of no_large
        no_large = random.randrange(0,5)
    # Select random small and large numbers
    large_random = random.sample([25,50,75,100], no_large)
    small_random = random.sample(list(range (1,11)), 6-no_large)
    # Play numbers
    play_numbers = large_random + small_random     
    # Pick a Target number
    target = random.randrange(101,1000)
    
    return play_numbers, target   

In [8]:
# Countdown numbers game, input for the solver
new_numbers_game()

([25, 100, 50, 75, 9, 3], 251)

## Countdown Game solver

In [9]:
# New Game
play_numbers, target = new_numbers_game()
play_numbers, target

([25, 75, 50, 1, 8, 3], 858)

In [10]:
for p in it.permutations(play_numbers,2):
    print(p)
    print(f'{p[0]}+{p[1]}={p[0]+p[1]}')
    print(f'{p[0]}*{p[1]}={p[0]*p[1]}')
    if p[0]-p[1]>0:
        print(f'{p[0]}-{p[1]}={p[0]-p[1]}')
    if p[0]/p[1] ==0:
        print(f'{p[0]}/{p[1]}={p[0]/p[1]}')
    print()

(25, 75)
25+75=100
25*75=1875

(25, 50)
25+50=75
25*50=1250

(25, 1)
25+1=26
25*1=25
25-1=24

(25, 8)
25+8=33
25*8=200
25-8=17

(25, 3)
25+3=28
25*3=75
25-3=22

(75, 25)
75+25=100
75*25=1875
75-25=50

(75, 50)
75+50=125
75*50=3750
75-50=25

(75, 1)
75+1=76
75*1=75
75-1=74

(75, 8)
75+8=83
75*8=600
75-8=67

(75, 3)
75+3=78
75*3=225
75-3=72

(50, 25)
50+25=75
50*25=1250
50-25=25

(50, 75)
50+75=125
50*75=3750

(50, 1)
50+1=51
50*1=50
50-1=49

(50, 8)
50+8=58
50*8=400
50-8=42

(50, 3)
50+3=53
50*3=150
50-3=47

(1, 25)
1+25=26
1*25=25

(1, 75)
1+75=76
1*75=75

(1, 50)
1+50=51
1*50=50

(1, 8)
1+8=9
1*8=8

(1, 3)
1+3=4
1*3=3

(8, 25)
8+25=33
8*25=200

(8, 75)
8+75=83
8*75=600

(8, 50)
8+50=58
8*50=400

(8, 1)
8+1=9
8*1=8
8-1=7

(8, 3)
8+3=11
8*3=24
8-3=5

(3, 25)
3+25=28
3*25=75

(3, 75)
3+75=78
3*75=225

(3, 50)
3+50=53
3*50=150

(3, 1)
3+1=4
3*1=3
3-1=2

(3, 8)
3+8=11
3*8=24



## Operators and functions

In [11]:
operator.add(4,5)

9

In [12]:
operator.mul(4,5)

20

In [13]:
operator.sub(4,5)

-1

In [14]:
operator.truediv(4,5)

0.8

In [15]:
ops = [operator.add, operator.sub, operator.mul, operator.truediv] *5
ops

[<function _operator.add(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.truediv(a, b, /)>]

In [16]:
# Patterns of calculations that may be used in Countdown Game
limit = 100
for q in it.permutations(ops,5):
    if limit == 0:
        break
    print(q)
    limit = limit -1

(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function truediv>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function sub>, <built-in function mul>, <built-in 

In [17]:
# Example of combination
L = [1,2,3,4]

for c in it.combinations(L,2):
    print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [18]:
# Example of permuation of size 2
L = [1,2,3,4]

for c in it.permutations(L,2):
    print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 4)
(4, 1)
(4, 2)
(4, 3)


In [19]:
# Example of permutation with repetition of size 2
L = [1,2,3,4]
for c in it.product(L,repeat = 2):
    print(c)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


In [20]:
# Using product
ops = [operator.add, operator.sub, operator.mul, operator.truediv]
limit = 100
for q in it.product(ops,repeat=5):
    if limit == 0:
        break
    print(q)
    limit = limit -1

(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in fun

# Reverse Polish Notation
**RPN** - is a mathematical notation in which operators follow their operands [[2](#section2)]
***

In [21]:
# Reverse Polish Notation
# 3 4 5 + *

## References
***

<a id='section1'></a>[[1] Countdown Game Example](https://datagenetics.com/blog/august32014/index.html)

<a id='section2'></a>[[2] Reverse Polish Notation ](https://en.wikipedia.org/wiki/Reverse_Polish_notation)